Problem 1
You are given 2 datasets
1. cc_info.csv – contains credit card numbers, their city, state, zip code and limit
2. transactions.csv- contains a random set of transactions made by those credit cards. Each transaction has a date, amount and coordinates where they occurred
Use both datasets to answer the following questions (if you are using SQL, consider them as tables, if you are using pandas, consider them as dataframes)
- 1 Which credit card had the highest purchase amount in each month in the dataset?
- 2 Which city has the biggest count of credit cards that exceeded (or reached) their limits?
- 3 If we assume that people only use their credit cards within their cities, can you figure out which transactions are fraudulent? Meaning that they took place in an “unusual” location relative to the card.

In [2]:
import pandas as pd
import numpy as np

In [6]:
transactions = pd.read_csv("E:\\Technologies\\my_python_codes\\mlcourse_dubai\\data\\transactions.csv")

In [14]:
transactions.head()

,credit_card,date,transaction_dollar_amount,Long,Lat
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719


In [15]:
transactions.shape

(294588, 5)

In [11]:
cc_info = pd.read_csv("E:\\Technologies\\my_python_codes\\mlcourse_dubai\\data\\cc_info.csv")

In [16]:
cc_info.head()

,credit_card,city,state,zipcode,credit_card_limit
0,1280981422329509,Dallas,PA,18612,6000
1,9737219864179988,Houston,PA,15342,16000
2,4749889059323202,Auburn,MA,1501,14000
3,9591503562024072,Orlando,WV,26412,18000
4,2095640259001271,New York,NY,10001,20000


In [17]:
cc_info.shape

(984, 5)

### Which credit card had the highest purchase amount in each month in the dataset?

In [37]:
transactions['year_month'] = transactions['date'].str[:7]

In [38]:
transactions

,credit_card,date,transaction_dollar_amount,Long,Lat,year_month
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370,2015-09
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114,2015-10
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004,2015-10
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895,2015-10
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719,2015-10
...,...,...,...,...,...,...
294583,9999757432802760,2015-09-10 19:43:33,127.23,-82.360952,32.978497,2015-09
294584,9999757432802760,2015-08-06 21:00:13,84.90,-82.322721,33.061071,2015-08
294585,9999757432802760,2015-09-22 16:15:47,77.54,-82.442130,32.949983,2015-09
294586,9999757432802760,2015-08-27 18:08:24,144.05,-82.398587,32.976162,2015-08


In [109]:
cc_monthly_trans = transactions.groupby(['year_month','credit_card'])['transaction_dollar_amount'].agg([sum])
idx = cc_monthly_trans.groupby(['year_month'])['sum'].transform(max)==cc_monthly_trans['sum']
cc_monthly_trans[idx].rename(columns = {'sum': 'Total_Spend'}, inplace = False).reset_index()

,year_month,credit_card,Total_Spend
0,2015-07,3787622376078564,2695.13
1,2015-08,5644143320838397,50080.47
2,2015-09,5644143320838397,44157.64
3,2015-10,5644143320838397,51569.82


### Which city has the biggest count of credit cards that exceeded (or reached) their limits?

In [279]:
cc_spend = transactions.groupby(['credit_card'])['transaction_dollar_amount'].agg([sum]).\
                rename(columns = {'sum': 'Total_Spend'}, inplace = False).reset_index()

In [280]:
cc_all =pd.merge(cc_spend, cc_info, on='credit_card', how='inner')

In [281]:
cc_all

,credit_card,Total_Spend,city,state,zipcode,credit_card_limit
0,1003715054175576,28839.84,Houston,PA,15342,20000
1,1013870087888817,36814.88,Washington,NH,3280,15000
2,1023820165155391,61052.56,Charlotte,VT,5445,28000
3,1073931538936472,7406.27,Dallas,PA,18612,10000
4,1077622576192810,1790.73,Houston,PA,15342,10000
...,...,...,...,...,...,...
979,9958678964376192,17471.96,Washington,NH,3280,10000
980,9961694231875562,14572.15,Washington,NH,3280,10000
981,9981251982982618,39008.71,New York,NY,10001,18000
982,9986135779184360,34517.33,Houston,PA,15342,14000


In [282]:
cc_all['city'][cc_all['Total_Spend'] >= cc_all['credit_card_limit']].value_counts().sort_values(ascending=False).head(1)

Washington    323
Name: city, dtype: int64

### If we assume that people only use their credit cards within their cities, can you figure out which transactions are fraudulent? Meaning that they took place in an “unusual” location relative to the card.

In [329]:
data = pd.merge(transactions,cc_info, on='credit_card', how='inner')

In [342]:
data.head(100)

,credit_card,date,transaction_dollar_amount,Long,Lat,year_month,city,state,zipcode,credit_card_limit
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370,2015-09,Houston,PA,15342,20000
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114,2015-10,Houston,PA,15342,20000
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004,2015-10,Houston,PA,15342,20000
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895,2015-10,Houston,PA,15342,20000
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719,2015-10,Houston,PA,15342,20000
...,...,...,...,...,...,...,...,...,...,...
95,1003715054175576,2015-08-19 17:28:27,120.93,-80.261018,40.294456,2015-08,Houston,PA,15342,20000
96,1003715054175576,2015-10-29 18:07:29,55.33,-80.143512,40.257870,2015-10,Houston,PA,15342,20000
97,1003715054175576,2015-10-12 21:48:15,95.09,-80.128982,40.254279,2015-10,Houston,PA,15342,20000
98,1003715054175576,2015-09-16 23:30:50,77.53,-80.167721,40.249845,2015-09,Houston,PA,15342,20000


In [345]:
pd.set_option('display.max_columns', None)

In [366]:
data[data['credit_card']==1003715054175576].head(260)

,credit_card,date,transaction_dollar_amount,Long,Lat,year_month,city,state,zipcode,credit_card_limit
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370,2015-09,Houston,PA,15342,20000
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114,2015-10,Houston,PA,15342,20000
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004,2015-10,Houston,PA,15342,20000
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895,2015-10,Houston,PA,15342,20000
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719,2015-10,Houston,PA,15342,20000
...,...,...,...,...,...,...,...,...,...,...
255,1003715054175576,2015-09-02 20:54:00,905.54,-80.237126,40.290891,2015-09,Houston,PA,15342,20000
256,1003715054175576,2015-10-02 15:55:48,57.50,-80.241273,40.193112,2015-10,Houston,PA,15342,20000
257,1003715054175576,2015-10-07 01:16:12,0.97,-80.196676,40.225513,2015-10,Houston,PA,15342,20000
258,1003715054175576,2015-08-07 02:28:31,71.02,-80.283377,40.188913,2015-08,Houston,PA,15342,20000


In [365]:
transactions[transactions['credit_card']==1003715054175576]

,credit_card,date,transaction_dollar_amount,Long,Lat,year_month
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370,2015-09
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114,2015-10
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004,2015-10
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895,2015-10
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719,2015-10
...,...,...,...,...,...,...
256,1003715054175576,2015-10-02 15:55:48,57.50,-80.241273,40.193112,2015-10
257,1003715054175576,2015-10-07 01:16:12,0.97,-80.196676,40.225513,2015-10
258,1003715054175576,2015-08-07 02:28:31,71.02,-80.283377,40.188913,2015-08
259,1003715054175576,2015-10-29 23:47:59,103.11,8.360145,36.188123,2015-10


In [361]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
address ="Houston,PA,15342"
loc = geolocator.geocode(address)
print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)
location = geolocator.reverse('31.1355603'+","+'-85.2654592')
location
location = geolocator.reverse('40.267370'+","+'-80.174132')
location.raw

latitude is :- 40.2464593 
longtitude is:- -80.2114472


{'place_id': 91407299,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 15042631,
 'lat': '40.26730372852077',
 'lon': '-80.17408041498314',
 'display_name': 'Cedar Avenue, Canonsburg, Washington County, Pennsylvania, 15317, United States',
 'address': {'road': 'Cedar Avenue',
  'village': 'Canonsburg',
  'county': 'Washington County',
  'state': 'Pennsylvania',
  'postcode': '15317',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['40.264976', '40.2683666', '-80.1769724', '-80.1727619']}

In [355]:
cc_info[cc_info['credit_card']==1003715054175576]

,credit_card,city,state,zipcode,credit_card_limit
915,1003715054175576,Houston,PA,15342,20000


In [360]:
data['city']

AttributeError: 'Series' object has no attribute 'distinct'

In [302]:
from geopy.geocoders import Nominatim

def get_city(Longitude,Latitude):
    # initialize Nominatim API
    geolocator = Nominatim(user_agent="geoapiExercises") 
    print(Latitude+","+Longitude)
    location = geolocator.reverse(Latitude+","+Longitude)
    address = location.raw['address']
    city = address.get('village')
    #state = address.get('state', '')
    ##country = address.get('country', '')
    #code = address.get('country_code')
    #zipcode = address.get('postcode')
    return city

In [305]:
data[['Lat', 'Long']].astype(str).apply(lambda row: get_city(row.split(" ")),axis=1)

AttributeError: 'Series' object has no attribute 'split'

In [301]:
data1['trans'] = (data[['Lat', 'Long']].astype(str)
                   .apply(lambda row: city_state_country(', '.join(row)),
                          axis=1)
                 )

0    40.267370
1    40.180114
2    40.313004
3    40.290895
4    40.166719
Name: Lat, dtype: float64,0   -80.174132
1   -80.194240
2   -80.211033
3   -80.174138
4   -80.238720
Name: Long, dtype: float64


ValueError: Must be a coordinate pair or Point

In [311]:
def city_state_country(coord):
    try:
        location = geolocator.reverse(coord)
        address = location.raw['address']
        city = address.get('village', '')
        return city
    except ValueError:
        return(None)
    except NoneType:
        return(None)

In [312]:
data1 = data.head()
type(data1)

pandas.core.frame.DataFrame

In [315]:
data1

,credit_card,date,transaction_dollar_amount,Long,Lat,year_month,city,state,zipcode,credit_card_limit,trans_city
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370,2015-09,Houston,PA,15342,20000,Canonsburg
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114,2015-10,Houston,PA,15342,20000,
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004,2015-10,Houston,PA,15342,20000,
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895,2015-10,Houston,PA,15342,20000,
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719,2015-10,Houston,PA,15342,20000,


In [314]:
data1['trans_city'] = (data1[['Lat', 'Long']].astype(str).apply(lambda row: city_state_country(', '.join(row)),axis=1))

<ipython-input-314-6ce064160f27>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['trans_city'] = (data1[['Lat', 'Long']].astype(str).apply(lambda row: city_state_country(', '.join(row)),axis=1))


In [297]:
data1['trans'] = get_city(data1['Long'],data1['Lat'])

ValueError: Must be a coordinate pair or Point

In [267]:
data.apply(lambda x: get_city(x.Long, x.Lat), 1).head()

KeyboardInterrupt: 

In [229]:
from geopy.geocoders import Nominatim
# initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises") 
location = geolocator.reverse('-80.174132, 40.267370')
address = location.raw['address']
city = address.get('village')
print(city)

AttributeError: 'NoneType' object has no attribute 'raw'

In [328]:
data

{'place_id': 86316280,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 9349882,
 'boundingbox': ['33.052875', '33.057271', '-82.40634', '-82.397942'],
 'lat': '33.054824',
 'lon': '-82.401691',
 'display_name': 'Bridges Road, Jefferson County, Georgia, 30434, United States',
 'class': 'highway',
 'type': 'unclassified',
 'importance': 0.9199999999999999}

In [260]:
def city_state_country(coord):
    try:
        location = geolocator.reverse(coord)
        address = location.raw['address']
        city = address.get('village', '')
        return city
    except ValueError:
        return(None)
    except NoneType:
        return(None)

data['trans_city'] = (data[['Lat', 'Long']].astype(str)
                   .apply(lambda row: city_state_country(', '.join(row)),
                          axis=1)
                 ).head()

NameError: name 'NoneType' is not defined

In [291]:
location = geolocator.reverse('-80.174132, 40.267370', exactly_one=True)
address = location.raw['address']
city = address.get('city', '')

AttributeError: 'NoneType' object has no attribute 'raw'

In [327]:
data

{'place_id': 86316280,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 9349882,
 'boundingbox': ['33.052875', '33.057271', '-82.40634', '-82.397942'],
 'lat': '33.054824',
 'lon': '-82.401691',
 'display_name': 'Bridges Road, Jefferson County, Georgia, 30434, United States',
 'class': 'highway',
 'type': 'unclassified',
 'importance': 0.9199999999999999}

In [292]:
from geopy.geocoders import Nominatim
  
# initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")
  
# Latitude & Longitude input
Latitude = "40.2673703430082"
Longitude = "-80.1741323292432"
  
print(Latitude+","+Longitude)
location = geolocator.reverse(Latitude+","+Longitude)
  
address = location.raw['address']
  
# traverse the data
city = address.get('village')
address
location

40.2673703430082,-80.1741323292432


Location(Cedar Avenue, Canonsburg, Washington County, Pennsylvania, 15317, United States, (40.267303698348016, -80.1740804537462, 0.0))

In [317]:
transactions

,credit_card,date,transaction_dollar_amount,Long,Lat,year_month
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370,2015-09
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114,2015-10
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004,2015-10
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895,2015-10
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719,2015-10
...,...,...,...,...,...,...
294583,9999757432802760,2015-09-10 19:43:33,127.23,-82.360952,32.978497,2015-09
294584,9999757432802760,2015-08-06 21:00:13,84.90,-82.322721,33.061071,2015-08
294585,9999757432802760,2015-09-22 16:15:47,77.54,-82.442130,32.949983,2015-09
294586,9999757432802760,2015-08-27 18:08:24,144.05,-82.398587,32.976162,2015-08


In [325]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="test")
location = geolocator.reverse("33.068351, -82.389680")
#print(location.address)
#print(location.raw)
location = geolocator.geocode(location.raw['display_name'])
print(location.raw)
data = location.raw
loc_data = data['display_name'].split()
print("Full Location")
print(loc_data)
print("Zip code : ",loc_data[-2])

{'place_id': 86316280, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 9349882, 'boundingbox': ['33.052875', '33.057271', '-82.40634', '-82.397942'], 'lat': '33.054824', 'lon': '-82.401691', 'display_name': 'Bridges Road, Jefferson County, Georgia, 30434, United States', 'class': 'highway', 'type': 'unclassified', 'importance': 0.9199999999999999}
Full Location
['Bridges', 'Road,', 'Jefferson', 'County,', 'Georgia,', '30434,', 'United', 'States']
Zip code :  United
